In [ ]:
import matplotlib.pylab as plt
import pandas as pd
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import numpy as np
import hdbscan
import socket
import struct

In [ ]:
DS1 = "./csv/convert/capture1_1.csv"
DSMERGED = "./csv/convert/capture_merge_1.csv"
LOG = "./csv/convert/conn.log.labeled"
# turn this to true, to skip all unnecessary fields
FAST = True

# Datensatzauswahl

Welche Datensätze haben wir in betracht gezogen und warum haben wir uns für diesen entschieden?

# Vorstellung der .log files und deren Struktur 
### Beispielhaft für Datensatz 1

In [ ]:
fields=["ts","uid","orig_h","orig_p","resp_h","resp_p","proto","service","duration","orig_bytes","resp_bytes","conn_state","local_orig","local_resp","missed_bytes","history","orig_pkts","orig_ip_bytes","resp_pkts","resp_ip_bytes","tunnel_parents","label","detailed-label"]

df_raw_log = pd.read_csv(LOG,skipfooter=1, engine="python", names=fields, sep="\x09|\x20\x20\x20", skiprows=8)
df_raw_log = df_raw_log.replace('-', np.nan)


In [ ]:
if not FAST:
    pd.set_option('display.max_columns', None)
    df_raw_log.head()

### Spaltenübersicht

<div style="tposition>
    
| Spalte Original   | Beschreibung                      | behalten  | 
|-------------------|-----------------------------------|-----------|
| ts		        | Zeitstempel 			            | X         |
| uid		        | UUID 			                    | X         |
| orig_h		    | IP sender adresse  			    | X         |
| orig_p		    | Sender port 			            | X         |
| resp_h		    | IP Empfänger addresse 			| X         |
| resp_p		    | Empfänger port 			        | X         |
| proto		        | Protokoll 			            | X         |
| service		    | Dienst 			                | X         |
| duration		    | Dauer 			                | X         |
| orig_bytes		| Ursprüngliche Bytes 		        | X         |
| resp_bytes		| Antwortbytes 			            | X         |
| conn_state		| Verbindungszustand 		        | X         |
| local_orig		| Lokal (Ursprung) 		            | X         |
| local_resp		| Lokal (Antwort) 		            | X         |
| missed_bytes	    | Verpasste Bytes 		            | X         |
| history		    | Historie 			                | X         |
| orig_pkts		    | Ursprüngliche Pakete 		        | X         |
| orig_ip_bytes	    | Ursprüngliche IP-Bytes 	        | X         |
| resp_pkts		    | Antwortpakete 			        | X         |
| resp_ip_bytes	    | Antwort IP-Bytes 		            | X         |
| tunnel_parents	| Tunnel-Eltern 			        | X         |
| label		        | Gut oder Schlecht 		        | X         |
| detailed-label	| Angriffsmethode 		            | X         |
</div>

In [ ]:
if not FAST:
    df_raw_log.columns

In [ ]:
if not FAST:
    df_raw_log.isna().sum()

| Spalte Original   | Beschreibung                      | behalten  | 
|-------------------|-----------------------------------|-----------|
| ts		        | Zeitstempel 			            | X         |
| uid		        | UUID 			                    | X         |
| orig_h		    | IP sender adresse  			    | X         |
| orig_p		    | Sender port 			            | X         |
| resp_h		    | IP Empfänger addresse 			| X         |
| resp_p		    | Empfänger port 			        | X         |
| proto		        | Protokoll 			            | X         |
| service		    | Dienst 			                | X         |
| duration		    | Dauer 			                | X         |
| orig_bytes		| Ursprüngliche Bytes 		        | X         |
| resp_bytes		| Antwortbytes 			            | X         |
| conn_state		| Verbindungszustand 		        | X         |
| local_orig		| Lokal (Ursprung) 		            | X         |
| local_resp		| Lokal (Antwort) 		            | X         |
| missed_bytes	    | Verpasste Bytes 		            | X         |
| history		    | Historie 			                | X         |
| orig_pkts		    | Ursprüngliche Pakete 		        | X         |
| orig_ip_bytes	    | Ursprüngliche IP-Bytes 	        | X         |
| resp_pkts		    | Antwortpakete 			        | X         |
| resp_ip_bytes	    | Antwort IP-Bytes 		            | X         |
| tunnel_parents	| Tunnel-Eltern 			        | X         |
| label		        | Gut oder Schlecht 		        | X         |
| detailed-label	| Angriffsmethode 		            | X         |


## Encoding der Spalten

In [ ]:
protos = list(df_raw_log.proto.unique().flatten())
conn_states = list(df_raw_log.conn_state.unique().flatten())
histories = list(df_raw_log.history.unique().flatten())
def convert_ipv4(addr):
    """Konvertieren einer IPv4 Adresse zu einem INT"""
    return struct.unpack("!I", socket.inet_aton(addr))[0]

def con_proto(proto):
    """Konvertieren der Ports ICMP, TCP und UDP zu einem INT"""
    return protos.index(proto)

def convert_connstate(state):
    return conn_states.index(state)

def convert_histories(history):
    return histories.index(history)
        

### Konvertieren des DataFrames in ein normiertes Format
1. Konvertieren der IPv4 Spalten
2. Konvertieren der Protokoll Spalte
3. Löschen der ts und uid spalten

In [ ]:
def norm_df(df):
    """
    Normierung des DataFrames, dh löschen der Spalten, welche nicht gebraucht werden und konvertieren der object Spalten zu INT
    """
    df = df.replace('-', np.nan)
    df['orig_h'] = df['orig_h'].apply(convert_ipv4)
    df['resp_h'] = df['resp_h'].apply(convert_ipv4)
    df['proto'] = df['proto'].apply(con_proto)
    df['conn_state'] = df['conn_state'].apply(convert_connstate)
    df['history'] = df['history'].apply(convert_histories)
    df = df.drop(['uid', 'ts', 'tunnel_parents', 'detailed-label', 'service', 'duration', 'orig_bytes', 'resp_bytes', 'local_orig', 'local_resp'], axis=1)
    df['label'] = (df['label'] == "Malicious").astype(int)
    df = df.dropna()
    df = df.reset_index(drop=True)
    return df

In [ ]:
df_conversion_1 = norm_df(df_raw_log)

In [ ]:
if not FAST:
    df_conversion_1.dtypes

In [ ]:
if not FAST:
    df_conversion_1.head(20)

In [ ]:
if not FAST:
    df_conversion_1.label.value_counts()

In [ ]:
def viz(df):
    for c, label in enumerate(df.columns):
        counts, edges, bars = plt.hist(df[df["label"] == 1][label], color='blue', label='Malicious', alpha=0.7, density=True, stacked=True)
        counts2, edges2, bars2 = plt.hist(df[df["label"] ==  0][label], color='red', label='Benign', alpha=0.7, density=True, stacked=True)
        plt.title(label)
        plt.ylabel("probability density")
        plt.xlabel(label)
        plt.legend()
        plt.show()

In [ ]:
if not FAST:
    viz(df_conversion_1)

## Scatter plot der Daten

In [ ]:
def make_bytes_plot(df) -> plt.figure:
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(12,12))
    sns.scatterplot(data=df, x='orig_p', y='resp_p', hue='label', ax=ax1)
    sns.scatterplot(data=df, x='orig_p', y='resp_p', hue='proto', ax=ax3)
    sns.scatterplot(data=df, x='orig_ip_bytes', y='resp_ip_bytes', hue='label', ax=ax2)
    sns.scatterplot(data=df, x='orig_ip_bytes', y='resp_ip_bytes', hue='proto', ax=ax4)
    plt.show()
    return fig

In [ ]:
if not FAST:
    make_bytes_plot(df_conversion_1)